In [3]:
'''
all written function are here 

'''

'\nall written function are here \n\n'

In [18]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.naive_bayes import GaussianNB

from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
# cleaning and pre processing data

dataPath = "data.csv"
data = pd.read_csv(dataPath)

# seperating features
label_column = 'Label' 
features = data.drop(columns=[label_column]).select_dtypes(include=['float64', 'int64'])
Labels = data[label_column]
# Add this to see excluded columns
excluded_columns = set(data.columns) - set(features.columns)
print("Excluded columns (including label):", excluded_columns)

#standardize
scaler = StandardScaler()
data_scaled = scaler.fit_transform(features)

Excluded columns (including label): {'DstMac', 'SrcMac', 'SrcAddr', 'Label', 'Sport', 'Dir', 'Flgs', 'DstAddr', 'Attack Category'}


In [6]:
def print_classification_results(confusion_matrix, classification_report, accuracy):
    #prints the cm results, classification report, and accuracy
    tn, fp, fn, tp = confusion_matrix.ravel()
    total = tn + fp + fn + tp
    
    # Calculate percentages
    tn_pct = (tn/total) * 100
    fp_pct = (fp/total) * 100
    fn_pct = (fn/total) * 100
    tp_pct = (tp/total) * 100
    
    # Print confusion matrix results
    print("\nConfusion Matrix Results:")
    print(f"""
    True Negatives: {tn} ({tn_pct:.2f}%)
    False Positives: {fp} ({fp_pct:.2f}%)
    False Negatives: {fn} ({fn_pct:.2f}%)
    True Positives: {tp} ({tp_pct:.2f}%)
    """)
    
    # Print classification report
    print("\nClassification Report:")
    print(classification_report)
    
    # Print accuracy
    print(f"\nModel Accuracy: {accuracy:.2%}")


In [7]:
def find_top_n_features(data_scaled, features, n):
    
    # -----------------applying PCA-----------------------
    pca = PCA(n_components=n)
    pca.fit(data_scaled)
    
    # Getting the top n features based on explained variance
    components = pca.components_
    #doesprint(components)
    explained_variance = pca.explained_variance_ratio_
    
    feature_importance = pd.DataFrame()
    for i in range(n):
        feature_importance[f'PC{i+1}'] = abs(components[i])
    
    feature_importance.index = features.columns
    
    # Getting overall importance by summing across components
    feature_importance['Overall_Importance'] = feature_importance.sum(axis=1)
    feature_importance = feature_importance.sort_values('Overall_Importance', ascending=False)
    
    # results
    print("\nFeature Importance by Principal Component:")
    pd.set_option('display.float_format', lambda x: '%.3f' % x)  # Format to 3 decimal places
    print(feature_importance)
    
    
    print(f"Total explained variance ratio: {sum(explained_variance):.2%}")
    print("\nExplained variance ratio by component:")
    for i, var in enumerate(explained_variance):
        print(f"PC{i+1}: {var:.2%}")
    
    print(f"\nTop {n} most important features:")
    top_n_features = feature_importance['Overall_Importance'].head(10)
    print(top_n_features)
    
    # printing out the combinations
    print("\nPrincipal Component Linear Combinations:")
    
    for pc in range(3): #only want to print the top three
        print(f"\nPC{pc+1} = ", end="")
        # Get the components for this PC
        coefficients = components[pc]
        terms = []
        for feat, coef in zip(features.columns, coefficients):
            if abs(coef) > 0.1: #only print if higher than .1
                terms.append(f"({coef:.3f} × {feat})")
        print(" + ".join(terms))
    
    return feature_importance, top_n_features

In [27]:
def perform_kmeans_clustering(data_scaled, features, labels, n_components=2):
    """
    Perform K-means clustering on PCA components using selected features
    
    Args:
        data_scaled: scaled numpy array from StandardScaler
        features: DataFrame with only the features to analyze
        labels: target labels
        n_components: number of PCA components to use
    """
    print("\nperformin k means on this many features: ", len(features.columns))
    
    
    # Convert data_scaled to DataFrame with feature names
    scaled_df = pd.DataFrame(data_scaled, columns=features.columns)
    
    # Select only the specified features
    selected_features = scaled_df[features.columns]
    
    # PCA on selected features
    pca = PCA(n_components=n_components)
    pc_features = pca.fit_transform(selected_features)
    
    pca_df = pd.DataFrame(
        data=pc_features,
        columns=[f'PC{i+1}' for i in range(n_components)]
    )
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        pca_df, labels, test_size=0.1, random_state=42
    )
    
    # K-means clustering
    kmeans = KMeans(n_clusters=2, random_state=42)
    kmeans.fit(X_train)
    y_pred = kmeans.predict(X_test)
    
    # Evaluation metrics
    cm = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)    
    accuracy = accuracy_score(y_test, y_pred)    
    return cm, class_report, accuracy

In [28]:
cm, class_report, accuracy = perform_kmeans_clustering(data_scaled, features, data[label_column], n_components=3)

print_classification_results(cm, class_report, accuracy)


performin k means on this many features:  36

Confusion Matrix Results:

    True Negatives: 1436 (87.99%)
    False Positives: 2 (0.12%)
    False Negatives: 194 (11.89%)
    True Positives: 0 (0.00%)
    

Classification Report:
              precision    recall  f1-score   support

           0       0.88      1.00      0.94      1438
           1       0.00      0.00      0.00       194

    accuracy                           0.88      1632
   macro avg       0.44      0.50      0.47      1632
weighted avg       0.78      0.88      0.82      1632


Model Accuracy: 87.99%


In [10]:
# feature_importance, top_features = find_top_n_features(data_scaled, features, 10)


In [11]:
def train_random_forest(data_scaled, Labels, features, n_estimators):
    print(f"Training Random Forest with {n_estimators} estimators")

    X_train, X_test, y_train, y_test = train_test_split(data_scaled, Labels, test_size=0.3, random_state=42)

    rf_classifier = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
    rf_classifier.fit(X_train, y_train)

    # Make predictions
    y_pred = rf_classifier.predict(X_test)

    # Print classification report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    # Create confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)    


    feature_importance = pd.DataFrame({
        'feature': features.columns,
        'importance': rf_classifier.feature_importances_
    })
    feature_importance = feature_importance.sort_values('importance', ascending=False)

    print("\nTop 5 Most Important Features:")
    print(feature_importance.head(5))

    # Calculate and print accuracy
    accuracy = accuracy_score(y_test, y_pred)    

    
    return cm, class_report, accuracy 

In [12]:
forest_cm, forest_report, forest_accuracy = train_random_forest(data_scaled, Labels, features, 100)

print_classification_results(forest_cm, forest_report, forest_accuracy)

Training Random Forest with 100 estimators

Classification Report:
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      4252
           1       0.97      0.51      0.67       644

    accuracy                           0.93      4896
   macro avg       0.95      0.76      0.82      4896
weighted avg       0.94      0.93      0.92      4896


Top 5 Most Important Features:
      feature  importance
8     DIntPkt    0.133990
21       Load    0.096434
12  DstJitter    0.093912
17        Dur    0.086107
3     SrcLoad    0.082480

Confusion Matrix Results:

    True Negatives: 4241 (86.62%)
    False Positives: 11 (0.22%)
    False Negatives: 313 (6.39%)
    True Positives: 331 (6.76%)
    

Classification Report:
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      4252
           1       0.97      0.51      0.67       644

    accuracy                           0.93      4896
   macro avg

In [ ]:
def perform_lda(features_scaled, labels):
    """
    Perform Linear Discriminant Analysis using numeric features.
    
    Args:
        dataPath (str): Path to the CSV data file
    Returns:
        tuple: (accuracy, predictions, actual_values)
    """
    #put in the features scaled and the labels we want to predict
    
    # Split using the features DataFrame
    X_train, X_test, y_train, y_test = train_test_split(
        features_scaled, labels, test_size=0.3, random_state=42
    )

    # Initialize and fit LDA model
    lda = LDA()
    lda.fit(X_train, y_train)

    # Print LDA components information
    print("\nLDA Components Information:")
    print(f"Number of components: {lda.n_components}")
    print("\nExplained variance ratio:")
    print(lda.explained_variance_ratio_)
    
    # Print component coefficients
    print("\nLinear Discriminant Coefficients:")
    for i, component in enumerate(lda.coef_):
        print(f"\nLD{i+1} coefficients:")
        for feat, coef in zip(features.columns, component):
            print(f"{feat}: {coef:.4f}")

    # Make predictions
    y_pred = lda.predict(X_test)
    # Calculate confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print("\nConfusion Matrix:")
    print(conf_matrix)

    # Generate classification report
    class_report = classification_report(y_test, y_pred)
    print("\nClassification Report:")
    print(class_report)
    accuracy = accuracy_score(y_test, y_pred)   
    return conf_matrix, class_report, accuracy

In [17]:
cm, report, accuracy = perform_lda(data_scaled, Labels)

print_classification_results(cm, report, accuracy)


LDA Components Information:
Number of components: None

Explained variance ratio:
[1.]

Linear Discriminant Coefficients:

LD1 coefficients:
Dport: 0.0000
SrcBytes: 4.5746
DstBytes: -9.3746
SrcLoad: -0.4146
DstLoad: -3.5580
SrcGap: -0.0000
DstGap: 0.0000
SIntPkt: -2.3615
DIntPkt: -0.4477
SIntPktAct: 453.4666
DIntPktAct: -0.0000
SrcJitter: -153.6353
DstJitter: 7.8913
sMaxPktSz: 0.0185
dMaxPktSz: -0.9048
sMinPktSz: -0.7674
dMinPktSz: 0.0000
Dur: 2.0393
Trans: 0.0000
TotPkts: 2.6564
TotBytes: -1.0525
Load: -2.3268
Loss: -552.0637
pLoss: 1173.2676
pSrcLoss: -384.6796
pDstLoss: -600.8920
Rate: 4.7021
Packet_num: -0.0375
Temp: -0.0457
SpO2: 0.0027
Pulse_Rate: 0.2497
SYS: -0.0287
DIA: 0.1639
Heart_rate: 0.0998
Resp_Rate: 0.0286
ST: 0.2382

Confusion Matrix:
[[4233   19]
 [ 351  293]]

Classification Report:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      4252
           1       0.94      0.45      0.61       644

    accuracy           

In [ ]:
def gaussian_naive_bayes(data_scaled, labels, features):
    """
    Perform Gaussian Naive Bayes classification on scaled data
    
    Args:
        data_scaled: scaled numpy array from StandardScaler
        labels: target labels
        features: original features DataFrame for column names
    Returns:
        tuple: (confusion_matrix, classification_report, accuracy)
    """
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        data_scaled, labels, test_size=0.3, random_state=42
    )
    
    # Initialize and fit GNB model
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
    
    # Make predictions
    y_pred = gnb.predict(X_test)
    
    # Calculate metrics
    cm = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    
    return cm, class_report, accuracy



In [ ]:
# Usage:
cm, report, acc = gaussian_naive_bayes(data_scaled, Labels, features)
print_classification_results(cm, report, acc)